In [33]:
import pandas as pd
import os
_g = globals()
import numpy as np

In [34]:
directory='/home/xgl470/energy/'

In [35]:
pd.set_option('display.max_columns', None)
import sys  

#reload(sys)  
#sys.setdefaultencoding('utf8')

In [36]:
def standardize_API_col(df):
    column_list=list(df.columns)
    new=[]
    for x in column_list:
        new_entry=x
        if x.upper().startswith("API"):
            new_entry='APINumber'
        new.append(new_entry)
    return new

# pull in Original Datasets

In [37]:
#North Dakota
nd_df=pd.read_csv(directory+"nd_data/nd_final_dataset.txt", sep='|')
nd_df.columns=standardize_API_col(nd_df)
nd_df = nd_df.drop('Unnamed: 0', 1)
nd_df[['APINumber']]=nd_df.apply(lambda x: '{0:20.0f}'.format(x.APINumber).strip(),axis=1)

In [38]:
#Frac Focus ingredients/purpose
ff_ingredient_df=pd.read_csv(directory+"frac_focus_data/ingredient_purpose_final.txt", sep='|')
ff_ingredient_df.columns=standardize_API_col(ff_ingredient_df)
ff_ingredient_df = ff_ingredient_df.drop('Unnamed: 0', 1)

In [39]:
#Frac Focus
ff_reg_df=pd.read_csv(directory+"frac_focus_data/registry_upload_final.txt", sep='|')
ff_reg_df.columns=standardize_API_col(ff_reg_df)
ff_reg_df = ff_reg_df.drop('Unnamed: 0', 1)
ff_reg_df[['APINumber']]=ff_reg_df.apply(lambda x: '{0:20.0f}'.format(x.APINumber).strip(),axis=1)

### create an api column for the FF ingredients df

In [40]:
key_api=ff_reg_df[['pKey','APINumber']]
key_api = key_api.drop_duplicates(subset='pKey')

In [41]:
ff_ingredient_df=pd.merge(ff_ingredient_df, key_api,how='inner', on=['pKey'])

In [42]:
temp='/home/xgl470/energy/texas_rrc_data/'
files=os.listdir(temp)
files=[x for x in files if '.csv' in x]
#files=[x for x in files if 'G-2' in x or 'W-2' in x]
files.sort()

### create an API and Well No. column in each Texas df

In [43]:
def TX_API_Col(x):
    try:
        return int('42'+str(x).strip("0"))
    except:
        return None

In [44]:
#Texas Data
final_files=[]
for x in files:
    char=x.replace('.csv','')
    char=char.replace('-','_')
    final_files.append(char)
    _g[char]=pd.read_csv(directory+"texas_rrc_data/"+x, sep='|')

In [45]:
well_tracking_api=PACKET[['WELL_NO','TRACKING_NO','API_NO']]
well_tracking_api['API_NO']=well_tracking_api['API_NO'].apply(TX_API_Col)

/opt/anaconda/anaconda.2.2.0/lib/python2.7/site-packages/IPython/kernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from IPython.kernel.zmq import kernelapp as app


In [46]:
for x in files:
    char=x.replace('.csv','')
    char=char.replace('-','_')
    cols_to_use = list(well_tracking_api.columns - _g[char].columns)
    cols_to_use.append('TRACKING_NO')
    if 'TRACKING_NO' in _g[char].columns and 'API_NO' in cols_to_use:
        _g[char]=pd.merge(_g[char], well_tracking_api[cols_to_use],how='left', on=['TRACKING_NO'])
        _g[char]=_g[char].sort_index(axis=1)
        _g[char].to_csv(directory+'texas_rrc_data/'+"%s" %(x), sep='|')

### Texas Operator lookup

In [47]:
oper_lkp=pd.read_csv(directory+"texas_rrc_data/operator_lkp.csv", sep='|')
oper_lkp = oper_lkp.drop_duplicates(subset='OPERATOR_NO')

## combine Texas W2 and G1 files

In [48]:
def w2_g2_id(column):
    final=column
    if 'W2' in column:
        final=column.replace('W2','W2_G1')
    elif 'G1' in column:
        final=column.replace('G1','W2_G1')
    return final

In [49]:
def w2_g2_table(x):
    final=x
    if x=='W_2':
        final='W2_G1'
    else:
        final=x.replace('W_2_','G_1_')
    return final

In [50]:
for x in final_files:
    if ('W_2_' in x or  'G_1_' in x) or (x=='G_1' or x=='W_2'):
        columns=list(_g[x].columns)
        columns=[w2_g2_id(y) for y in columns]
        _g[x].columns=columns
        cols = _g[x].columns.tolist()
        cols.sort()
        _g[x] = _g[x][cols]

In [51]:
for x in final_files:
    if  x=='W_2':
        new_x='W2_G1'
        combine='G_1'
        _g[new_x] =_g[x].append(_g[combine],ignore_index = True)
        _g[new_x].to_csv(directory+'texas_rrc_data/'+"%s.txt" %(new_x), sep='|')
    elif 'W_2_' in x:
        new_x=x.replace('W_2_','')
        combine=x.replace('W_2_','G_1_')
        _g[new_x] =_g[x].append(_g[combine],ignore_index = True)
        _g[new_x].to_csv(directory+'texas_rrc_data/'+"%s.txt" %(new_x), sep='|')

In [52]:
P_4_Gas_Oil_Gatherer_Data =P_4_Gas_Gatherer_Data.append(P_4_Oil_Gatherer_Data,ignore_index = True)
P_4_Gas_Oil_Gatherer_Data.to_csv(directory+"texas_rrc_data/P_4_Gas_Oil_Gatherer_Data.txt" , sep='|')

# ## time to create a master df with all API's to build Tableau off of

In [53]:
def API_Col(x):
    try:
        return int(str(x).strip("0"))
    except:
        return None

### Texas

In [54]:
master_tx=PACKET[['API_NO','LATITUDE','LONGITUDE','OIL_GAS_TYPE_CODE_VALUE','WELLBORE_PROFILE_VALUE','FIELD_NAME','WELL_NO','OPERATOR_NO']]

In [55]:
oper_lkp=oper_lkp[oper_lkp.columns[0:2]]
oper_lkp.columns=['OPERATOR_NO','OPERATOR_NAME']

In [56]:
master_tx = pd.merge(master_tx, oper_lkp,how='left', on=['OPERATOR_NO'])
master_tx = master_tx.drop('OPERATOR_NO', 1)

In [57]:
master_tx.columns=['API_NO','LATITUDE','LONGITUDE','OIL_GAS_TYPE','WELLBORE','FIELD_NAME','WELL_NO','OPERATOR']

In [58]:
master_tx=master_tx.sort_index(axis=1)

### North Dakota

In [59]:
master_nd=nd_df[['APINumber','LATITUDE','LONGITUDE','WELLTYPE','WELLBORE','FIELDNAME','CURRENTWELLNAME','CURRENTOPERATOR']]

In [60]:
master_nd['APINumber']=master_nd['APINumber'].apply(API_Col)

/opt/anaconda/anaconda.2.2.0/lib/python2.7/site-packages/IPython/kernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [61]:
master_nd.columns=['API_NO','LATITUDE','LONGITUDE','OIL_GAS_TYPE','WELLBORE','FIELD_NAME','WELL_NO','OPERATOR']

In [62]:
master_nd=master_nd.sort_index(axis=1)

### combine TX+ND

In [63]:
master_nd_tx =master_tx.append(master_nd,ignore_index = True)
master_nd_tx = master_nd_tx.sort(['LATITUDE','LONGITUDE','LATITUDE','WELL_NO','FIELD_NAME','WELLBORE'], ascending=True)
master_nd_tx = master_nd_tx.drop_duplicates(subset='API_NO')

### FracFocus

In [64]:
master_ff=ff_reg_df[['APINumber','Latitude','Longitude','WellName','WellName','OperatorName']]

In [65]:
master_ff['APINumber']=master_ff['APINumber'].apply(API_Col)

/opt/anaconda/anaconda.2.2.0/lib/python2.7/site-packages/IPython/kernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [66]:
master_ff.columns=['API_NO','LATITUDE','LONGITUDE','FIELD_NAME','WELL_NO','OPERATOR']

In [67]:
master_ff["OIL_GAS_TYPE"] = np.nan
master_ff["WELLBORE"] = np.nan

/opt/anaconda/anaconda.2.2.0/lib/python2.7/site-packages/IPython/kernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/opt/anaconda/anaconda.2.2.0/lib/python2.7/site-packages/IPython/kernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from IPython.kernel.zmq import kernelapp as app


In [68]:
master_ff=master_ff.sort_index(axis=1)

# Fixing missing values in ND/TX

### Latitude/Longitude

In [69]:
#missing latitude in ND/TX
LL_nd_tx=master_nd_tx[master_nd_tx['LATITUDE']<>master_nd_tx['LATITUDE']]
master_nd_tx=master_nd_tx[master_nd_tx['LATITUDE']==master_nd_tx['LATITUDE']]

In [70]:
#valid latitude in FF
LL_ff=master_ff[master_ff['LATITUDE']==master_ff['LATITUDE']]
LL_ff = LL_ff.drop_duplicates(subset='API_NO')
LL_ff = LL_ff[['API_NO','LATITUDE','LONGITUDE']]

In [71]:
#replace ND/TX Lat/Long values
keep_these=['API_NO','FIELD_NAME','OIL_GAS_TYPE','OPERATOR', 'WELLBORE','WELL_NO']
LL_nd_tx = pd.merge(LL_nd_tx[keep_these], LL_ff,how='left', on=['API_NO'])
LL_nd_tx=LL_nd_tx.sort_index(axis=1)
master_nd_tx =master_nd_tx.append(LL_nd_tx,ignore_index = True)
master_nd_tx = master_nd_tx.drop_duplicates(subset='API_NO')

### combine FF with TX/ND now

In [72]:
master_nd_tx =master_nd_tx.append(master_ff,ignore_index = True)
master_nd_tx = master_nd_tx.drop_duplicates(subset='API_NO')

In [73]:
master_nd_tx.to_csv(directory+"master_lkp.txt", sep='|')

In [75]:
PACKET[PACKET['TRACKING_NO']==145586]

,API_NO,APPROVED_DT,CMPL_OR_RECMPL_DATE,CMPL_PACKET_TYPE,CMPL_PACKET_TYPE_VALUE,COMPLETION_TYPE,COMPLETION_TYPE_VALUE,COUNTY,DATE,DIRECTION_NEAREST_TOWN,DISTRICT,DIST_DIRECTION_DIRECTION,DIST_DIRECTION_MILES,DP_DT,DP_NO,ELECTRIC_LOG_OTHER_DESC,ELECTRIC_LOG_RUN,ELECTRIC_LOG_RUN_VALUE,FIELD_NAME,FIELD_NO,FILING_METHOD_CODE,FILING_METHOD_CODE_VALUE,FRMR_GAS_ID_OR_OIL_LEASE,FRMR_GAS_ID_OR_OIL_LSE_NO,FRMR_LEASE_NAME,FRMR_OPERATOR_NAME,FRMR_WELL_NO,HORIZ_WELLBORE_TYPE_CODE,HORIZ_WELLBORE_TYPE_CODE_VALUE,LATITUDE,LATLONG_CODE,LATLONG_CODE_VALUE,LATLONG_OTHER,LEASE_NAME,LEASE_NO,LONGITUDE,NEAREST_WELL_DIST,NO_OF_PRODUCING_WELLS,OIL_GAS_TYPE_CODE,OIL_GAS_TYPE_CODE_VALUE,OPERATOR_NO,OTHER_PERMIT_DATE,OTHER_PERMIT_DSCRPTN,OTHER_PERMIT_NO,PACKET_ID,PURPOSE_OF_FILING,PURPOSE_OF_FILING_VALUE,RULE37_CASE_DT,RULE37_CASE_NO,SLTWTR_DSPSL_PRMT_DT,SLTWTR_DSPSL_PRMT_NO,SPUD_DT,SUBMISSION_TYPE,SUBMIT_DT,SURVEY,SURVEY_ABSTRACT,SURVEY_BLOCK,SURVEY_SECTION,TOTAL_ACRES,TRACKING_NO,TYPE,UNIT_NO,WELLBORE_PROFILE,WELLBORE_PROFILE_VALUE,WELL_NO,WELL_TYPE,WELL_TYPE_VALUE,WORKOVER_ROW_COUNT,WTR_INJCTN_PRMT_DT,WTR_INJCTN_PRMT_NO
488,12333707,NaN,08/28/2015,2,G-1,1,NEW WELL,123,11-03-2015,CUERO,02,NW,13.5,08/16/2014,791893,NaN,11,GAMMA RAY (MWD),EAGLEVILLE (EAGLE FORD-2),27135750,1,ONLINE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,KRAUSE UNIT A,10200,NaN,316,3,2,OIL WELL,109333,NaN,NaN,NaN,204998,1,INITIAL POTENTIAL,NaN,NaN,NaN,NaN,03/30/2015,Submitted,11/03/2015,"PATE, P",386,NaN,NaN,347.24,145586,PACKET,NaN,1,HORIZONTAL,3,1,PRODUCING,0,NaN,NaN
